In [46]:
# y'(x)= - 2∙y(x) + y^2(x)
exact_sol(x) = 2 / (exp(2*x) + 1)
exact_sol(x) = 2 / (3*exp(2*x) + 1)
taylor_series(x) = 1 - x + x^3/3 - 2*x^5/15 + 17*x^7/315
taylor_series(x) = 1/2 - (3*x)/4 + (3*x^2)/8 + x^3/16 - (5*x^4)/32 + (13*x^5)/320 + (77*x^6)/1920

taylor_series (generic function with 1 method)

In [47]:
using Polynomials, SpecialPolynomials, DifferentialEquations
n = 1
(N, h) = (10^n, 10.0^(-n))
x_0 = 0
y0 = 0.5
k_range = -2:N

function calc_exact(x_arr, exact)
    for k in k_range
        x_k = x_0 + k * h
        y_exact_k = exact_sol(x_k)
        push!(x_arr, x_k)
        push!(exact, y_exact_k)
    end
end

function call_adams(x_arr)
    ans = [y0]
    
    x_table = collect(Iterators.take(x_arr,5))
    y_table = taylor_series.(x_table)
    println(x_table)
    println(y_table)
    lagrange = fit(Lagrange, x_table, y_table)
    adl = integrate(lagrange)
    for k in 1:N-1
        push!(ans, ans[k] + adl(x_0 + (k + 1) * h) - adl(x_0 + k * h))
    end
    print(ans)
    return ans
end

function call_adams2()
    fun(y, p, t) = -2*y + y*y
    tspan = (x_0, x_0 + N * h)
    prob = ODEProblem(fun, y0, tspan, dt=h)
    sol = solve(prob, AB4(), reltol = 1e-8, saveat=h)
    return sol.u
end
    
function call_rk4()
    fun(y, p, t) = -2*y + y*y
    tspan = (x_0, x_0 + N * h)
    prob = ODEProblem(fun, y0, tspan)
    sol = solve(prob, Tsit5(), reltol = 1e-8, saveat=h)
    return sol.u
end

function euler()
    fun(y, p, t) = -2*y + y*y
    tspan = (x_0, x_0 + N * h)
    prob = ODEProblem(fun, y0, tspan, dt=h)
    sol = solve(prob, Euler(), reltol = 1e-8)
    return sol.u
end

function euler1()
    fun(y, p, t) = -2*y + y*y
    tspan = (x_0, x_0 + N * h)
    prob = ODEProblem(fun, y0, tspan)
    sol = solve(prob, Midpoint(), reltol = 1e-8, saveat=h)
    return sol.u
end

function euler2()
    fun(y, p, t) = -2*y + y*y
    tspan = (x_0, x_0 + N * h)
    prob = ODEProblem(fun, y0, tspan)
    sol = solve(prob, Trapezoid(), reltol = 1e-8, saveat=h)
    return sol.u
end

euler2 (generic function with 1 method)

In [48]:
exact = []
x_arr = Vector{Float64}()
calc_exact(x_arr, exact)
taylor_sol = taylor_series.(x_arr)
adams = call_adams2()
rk4 = call_rk4()
euler_sol = euler()
euler1_sol = euler1()
euler2_sol = euler2()

header = (["k", "x_k", "exact", "taylor", "abs err taylor", "adams", "runge-kutta", "euler", "euler1", "euler2"])
k = collect(-2:N)
data = hcat(k, x_arr, exact, taylor_sol, map(abs, taylor_sol .- exact), [fill(missing, 2); adams], [missing; missing; rk4], [missing; missing; euler_sol], [missing; missing; euler1_sol], [missing; missing; euler2_sol])

using PrettyTables
print(pretty_table(data, header=header))
# adams by first 5 taylor


┌────┬──────┬───────────┬──────────┬────────────────┬───────────┬─────────────┬───────────┬───────────┬───────────┐
│  k │  x_k │     exact │   taylor │ abs err taylor │     adams │ runge-kutta │     euler │    euler1 │    euler2 │
├────┼──────┼───────────┼──────────┼────────────────┼───────────┼─────────────┼───────────┼───────────┼───────────┤
│ -2 │ -0.2 │   0.66424 │  0.66424 │     3.79485e-7 │   missing │     missing │   missing │   missing │   missing │
│ -1 │ -0.1 │  0.578672 │ 0.578672 │     3.02616e-9 │   missing │     missing │   missing │   missing │   missing │
│  0 │  0.0 │       0.5 │      0.5 │            0.0 │       0.5 │         0.5 │       0.5 │       0.5 │       0.5 │
│  1 │  0.1 │  0.428797 │ 0.428797 │     3.07344e-9 │  0.428798 │    0.428797 │     0.425 │  0.428797 │  0.428798 │
│  2 │  0.2 │  0.365265 │ 0.365266 │     3.92171e-7 │  0.365266 │    0.365265 │  0.358063 │  0.365265 │  0.365263 │
│  3 │  0.3 │  0.309293 │   0.3093 │     6.63953e-6 │  0.309294 │    0.3

In [49]:
header = (["k", "x_k", "exact", "err taylor", "err adams", "runge-kutta", "err euler", "err euler1", "err euler2"])
k = collect(-2:N)
data = hcat(k, x_arr, exact,
    map(abs, taylor_sol .- exact),
    map(abs, [missing; missing; adams] .- exact),
    map(abs, [missing; missing; rk4] .- exact),
    map(abs, [missing; missing; euler_sol] .- exact),
    map(abs, [missing; missing; euler1_sol] .- exact),
    map(abs, [missing; missing; euler2_sol] .- exact))

using PrettyTables
print(pretty_table(data, header=header))

┌────┬──────┬───────────┬─────────────┬────────────┬─────────────┬────────────┬────────────┬────────────┐
│  k │  x_k │     exact │  err taylor │  err adams │ runge-kutta │  err euler │ err euler1 │ err euler2 │
├────┼──────┼───────────┼─────────────┼────────────┼─────────────┼────────────┼────────────┼────────────┤
│ -2 │ -0.2 │   0.66424 │  3.79485e-7 │    missing │     missing │    missing │    missing │    missing │
│ -1 │ -0.1 │  0.578672 │  3.02616e-9 │    missing │     missing │    missing │    missing │    missing │
│  0 │  0.0 │       0.5 │         0.0 │        0.0 │         0.0 │        0.0 │        0.0 │        0.0 │
│  1 │  0.1 │  0.428797 │  3.07344e-9 │ 2.57356e-7 │ 7.99634e-10 │ 0.00379732 │ 1.11154e-8 │ 3.73748e-7 │
│  2 │  0.2 │  0.365265 │  3.92171e-7 │ 5.16134e-7 │  1.42453e-9 │ 0.00720267 │ 2.46219e-8 │ 1.82576e-6 │
│  3 │  0.3 │  0.309293 │  6.63953e-6 │ 7.59728e-7 │ 4.17914e-10 │  0.0100223 │ 3.88337e-8 │ 4.38493e-6 │
│  4 │  0.4 │  0.260531 │  4.90336e-5 │  2.554

### 